In [2]:
import codecs
import glob
import os
import re
import time
try:
    import cPickle as pickle
except:
    import pickle

import pandas as pd
import numpy as np
from scipy import optimize
import scipy as sp
from scipy import stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
#import altair as alt

In [4]:
# # Load in and preprocess trust rating data

# tr_1s_df=pd.DataFrame()
# # fname='../week1_tasks/tr_1s//tr_1s_542473a4fdf99b691fb38455_SESSION_2020-04-05_22h04.04.669.csv' # for testing
# for fname in glob.iglob('../week1_tasks/tr_1s/tr_1s_*.csv'):
#     dat=pd.read_csv(fname)
#     dat.dropna(subset=['trialrace'], inplace=True)
#     dat['normedkey']=stats.zscore(dat.key_press)
#     tmpMeanRat=dat.groupby(['trialrace']).mean().key_press
#     tmpMeanRat.rename(dat.subject_id.iloc[0], inplace=True)    
#     tmpNormMeanRat=dat.groupby(['trialrace']).mean().normedkey
#     tmpNormMeanRat.rename(dat.subject_id.iloc[0], inplace=True)
#     tmpNormMeanRat=tmpNormMeanRat.to_frame().transpose()
#     tmpNormMeanRat.columns=['nm_'+col for col in tmpNormMeanRat.columns]
#     tmp_tr_1s=tmpMeanRat.to_frame().transpose().join(tmpNormMeanRat)
#     tmp_tr_1s.loc[dat.subject_id.iloc[0],'rt_lt_300']=np.sum(dat.rt<300)
#     tmp_tr_1s.loc[dat.subject_id.iloc[0],'fname']=fname
#     tr_1s_df=tr_1s_df.append(tmp_tr_1s)

    
# # Eliminate later runs of duplicates
# tr_1s_df=tr_1s_df[~tr_1s_df['fname'].isin([i for i in tr_1s_df[tr_1s_df.index.duplicated()].fname])]

In [21]:
subjs=[]
for fname in glob.iglob('*Tests_*.csv'):  #120 only
#for fname in glob.iglob('*Test_*.csv'):  #all other subs
    dat=pd.read_csv(fname)
    tmpdat=dat[pd.to_numeric(dat['GambleAdviceNum'], errors='coerce').isnull()]
    
    for i in tmpdat.GambleAdviceNum.dropna().index:
        dat.iloc[i,3]=dat.iloc[i,3]+','+dat.iloc[i,4]
        
        for j in np.arange(4,dat.shape[1]-1):
            dat.iloc[i,j]=dat.iloc[i,j+1]
            
    dat.dropna(subset=['Choice'], inplace=True) #drop trials with NaN in choice column - inplace means replace orig array with result
    dat=dat.iloc[:,:-2] #for sub 120 only
    #dat=dat.iloc[:,:-3]
    dat.columns=list(dat.columns[:95])+['gender','date','expName','psychopyVersion','frameRate']
    dat['response.keys']=dat['response.keys'].replace('2','0') #code choice to not take advice as 0 instead of 2
    dat['response.keys']=dat['response.keys'].replace('None','').apply(pd.to_numeric)
    dat['resp.keys']=dat['resp.keys'].replace('left','0') #safe
    dat['resp.keys']=dat['resp.keys'].replace('right','1') #gamble
    dat['resp.keys']=dat['resp.keys'].replace('None','').apply(pd.to_numeric)
    dat['key_resp_3.keys']=dat['key_resp_3.keys'].replace('left','0') #expert adviser
    dat['key_resp_3.keys']=dat['key_resp_3.keys'].replace('right','1') #novice adviser
    dat['response.rt']=dat['response.rt'].apply(pd.to_numeric)
    dat['Amount']=dat['Amount'].replace('Pictures/IA20.bmp','20')
    dat['Amount']=dat['Amount'].replace('Pictures/IA25.bmp','25')
    dat['Amount']=dat['Amount'].replace('Pictures/IA40.bmp','40')       
    dat['Amount']=dat['Amount'].replace('Pictures/IA50.bmp','50')
    dat['Amount']=dat['Amount'].replace('Pictures/IA75.bmp','75')
    dat['Amount']=dat['Amount'].replace('Pictures/IA80.bmp','80')
    dat['Amount']=dat['Amount'].replace('Pictures/IA100.bmp','100')       
    dat['Amount']=dat['Amount'].replace('Pictures/IA110.bmp','110')
    dat.to_csv('processed/'+fname[:-4]+'_proc.csv')
    
#     meanRT=dat['response.rt'].mean()
#     meanResp=dat['response.keys'].mean()
    
#     summ={'RT': meanRT,
#          'resp': meanResp}
    
#     subjs.append(summ)
    


In [14]:
dat['Amount'].replace('Pictures/IA50.bmp','50')

4                      50
5      Pictures/IA100.bmp
6                      50
7                      50
8      Pictures/IA100.bmp
9                      50
11                     50
12                     50
13      Pictures/IA25.bmp
14     Pictures/IA110.bmp
15     Pictures/IA100.bmp
16      Pictures/IA25.bmp
17     Pictures/IA100.bmp
18      Pictures/IA80.bmp
19      Pictures/IA25.bmp
20                     50
21      Pictures/IA80.bmp
22     Pictures/IA100.bmp
23      Pictures/IA75.bmp
24     Pictures/IA100.bmp
25      Pictures/IA75.bmp
26      Pictures/IA75.bmp
27      Pictures/IA75.bmp
28      Pictures/IA75.bmp
29      Pictures/IA40.bmp
30                     50
32                     50
33     Pictures/IA100.bmp
34                     50
35      Pictures/IA75.bmp
              ...        
189                    50
190                    50
191     Pictures/IA75.bmp
192    Pictures/IA110.bmp
193     Pictures/IA75.bmp
194    Pictures/IA100.bmp
195     Pictures/IA25.bmp
196     Pict

In [ ]:
tmp=pd.DataFrame()

In [ ]:
tmp.loc[fname[:3],'RT']=meanRT

In [ ]:
tmp.loc[fname[:3],'resp']=meanResp

In [11]:
fname

'119_Test_2020_Mar_04_1159_DF.csv'

In [20]:
dat.groupby(['key_resp_3.keys']).mean()['response.keys']

key_resp_3.keys
left     1.666667
right    1.800000
Name: response.keys, dtype: float64

In [22]:
dat.groupby(['GambleAdviceType']).mean()['response.keys']

GambleAdviceType
risk    1.732824
safe    1.608696
Name: response.keys, dtype: float64

In [17]:
# [print(i) for i in dat.columns]

In [36]:
dat.groupby(['GambleAdviceType']).mean()['response.keys']

GambleAdviceType
risk    1.582677
safe    1.556962
Name: response.keys, dtype: float64

In [134]:
# dat.rename(columns=
# .columns=list(dat.columns[:95])+['gender','date','expName','psychopyVersion','frameRate']